In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
import datetime

In [31]:
url = "https://karofilm.ru"
url_theaters = url + "/theatres"

In [3]:
def remove_all(string):
    pattern = re.compile(r'[А-Яа-яёЁ0-9 ]+')
    return pattern.findall(string)[0].strip()

In [4]:
def find_all_theaters_KARO(theatres):
    dicti = {}
    metro_class = 'cinemalist__cinema-item__metro__station-list__station-item'
    for theater in theatres:
        dicti[theater.findAll('h4')[0].text[1:]] = {
            'metro': [remove_all(i.text) for i in theater.findAll('li', class_=metro_class)], 
            'address': theater.findAll('p')[0].text.split('+')[0].strip(),
            'phone': '+' + theater.findAll('p')[0].text.split('+')[-1],
            'data-id': theater['data-id']
        }
    return dicti

In [5]:
r = requests.get(url_theaters)
if r.status_code == 200:
    soup = BeautifulSoup(r.text, "html.parser")
    theatres = soup.findAll('li', class_='cinemalist__cinema-item')
    karo_theatres = find_all_theaters_KARO(theatres)
else:
    print("Страница не найдена")

In [6]:
conn = sqlite3.connect("cinemabot.db")
cursor = conn.cursor()

In [7]:
cursor.execute("""CREATE TABLE IF NOT EXISTS brand(
                id integer PRIMARY KEY,
                name text NOT NULL
                )""")

In [8]:
cursor.execute("""CREATE TABLE IF NOT EXISTS brand(
                id integer PRIMARY KEY,
                name text NOT NULL
                )""")

In [9]:
cursor.execute("""CREATE TABLE IF NOT EXISTS cinema_halls(
                id integer PRIMARY KEY,
                brand_id integer Not NULL,
                name text NOT NULL,
                address text NOT NULL,
                metro text NOT NULL,
                phone text NOT NULL,
                FOREIGN KEY (brand_id) REFERENCES brand(id)
                )""")

In [10]:
cursor.execute("""CREATE TABLE IF NOT EXISTS cinemas(
                id integer PRIMARY KEY,
                name text NOT NULL,
                duration integer NOT NULL,
                genres text NOT NULL
                )""")

In [30]:
cursor.execute("""CREATE TABLE IF NOT EXISTS sessions(
                id integer PRIMARY KEY,
                cinema_id integer Not NULL,
                hall_id integer Not NULL,
                name text NOT NULL,
                date date NOT NULL,
                time time NOT NULL,
                price integer NOT NULL,
                FOREIGN KEY (cinema_id) REFERENCES cinemas(id),
                FOREIGN KEY (hall_id) REFERENCES cinema_halls(id),
                FOREIGN KEY (name) REFERENCES cinemas(id)
                )""")

In [12]:
cursor.execute("insert into brand values (1, 'КАРО')")
cursor.execute("insert into brand values (2, 'Синема Парк')")
cursor.execute("insert into brand values (3, 'Алмаз Синема')")
conn.commit()

In [13]:
cursor.execute('select * from brand').fetchall()

[(1, 'КАРО'), (2, 'Синема Парк'), (3, 'Алмаз Синема')]

In [14]:
for key, item in karo_theatres.items():
    elements=[1,key,item['address'],', '.join(item['metro']),item['phone']]
    cursor.execute("insert into cinema_halls (brand_id, name, address, metro, phone) values ({},'{}','{}','{}','{}')".format(*elements))
conn.commit()

In [15]:
r2 = requests.get(url)
if r2.status_code == 200:
    soup = BeautifulSoup(r2.text, 'html.parser')
    films = soup.findAll('div', class_ = 'afisha-item-inner')
    for film in films:
        film_name = film.findAll('h3', class_ = "afisha-title")[0].text
        film_duration = film.findAll('span', class_ = 'afisha-duration-time')[0].text
        try:
            film_genres = film.findAll('p', class_ = 'afisha-genre')[0].text
        except IndexError:
            film_genres = 'undefined'
        elements = [film_name, film_duration, film_genres]
        try:
            cursor.execute("insert into cinemas (name, duration, genres) values ('{}', '{}', '{}')".format(*elements))
        except sqlite3.OperationalError:
            continue
    conn.commit()
else:
    print('I think that page is not found')

In [32]:
def try_to_get_cinema_id(name, cinemas):
    for el in cinemas:
        if name == el[1]:
            return el[0]
    for el in cinemas:
        if (name in el[1]) or (el[1] in name):
            return el[0]

In [33]:
films_class = 'cinema-page-item__schedule__row'
table_class = 'cinema-page-item__schedule__row__board-row'
first_class = table_class + '__left'
second_class = table_class + '__right'
date_class = 'widget-select'
for theater in karo_theatres:
    dates = {}
    url_theater_id = url_theaters + '?id=' + karo_theatres[theater]['data-id']
    r3 = requests.get(url_theater_id)
    if r3.status_code == 200:
        date_parser = BeautifulSoup(r3.text,'html.parser')
        date_list = date_parser.findAll('select',class_ = date_class)[0]
        date_list = [i['data-id'] for i in date_list.findAll('option')]
        for date in date_list: 
            url_theater_id_date = url_theater_id + '&date=' + date
            r4 = requests.get(url_theater_id_date)
            session = {}
            if r4.status_code == 200:
                films_parser = BeautifulSoup(r4.text,'html.parser')
                films_list = films_parser.findAll('div', class_ = films_class)
                for film in films_list:
                    name = film.findAll('h3')
                    if name:
                        name = name[0].text.split(', ')
                        session_time = {}
                        session_time['age'] = name[1]
                        for i in film.findAll('div', class_ = table_class):
                            time_D = i.findAll('div', class_ = first_class)[0].text.strip()
                            time = i.findAll('div', class_ = second_class)[0].findAll('a')
                            time = [j.text for j in time]
                            session_time[time_D] = time
                            for time_element in time:
                                cinema_id = try_to_get_cinema_id(name[0], cursor.execute('select * from cinemas').fetchall())
                                if cinema_id == None:
                                    cinema_id = 0
                                hall_id = cursor.execute(f'select id from cinema_halls where name=\'{theater}\'').fetchall()[0][0]
                                values = [cinema_id, hall_id, name[0], date, time_element, 0]
                                try:
                                    cursor.execute("insert into sessions (cinema_id, hall_id, name, date, time, price) values ({},{},'{}','{}', '{}', {})".format(*values))
                                except sqlite3.OperationalError:
                                    continue
                        session[name[0]] = session_time
            else:
                print('No date, url=', url_theater_id_date)
            dates[date]=session
    else:
        print('No cinema, url=', url_theater_id)
    karo_theatres[theater]['dates'] = dates
conn.commit()

In [34]:
url = "https://kinoteatr.ru"
url_theatres = url + "/raspisanie-kinoteatrov"
url_films = url + "/kinoafisha"

In [35]:
def tab_delete_name(name):
    pattern = re.compile(r'\w+[:,.!?]?')
    res = ' '.join(pattern.findall(name))
    return res

def date_maker(date):
    pattern = re.compile(r'[0-9]+-[0-9]+-[0-9]+')
    res = ', '.join(pattern.findall(date))
    return res

In [23]:
r = requests.get(url_theatres)
if r.status_code == 200:
    soup = BeautifulSoup(r.text, "html.parser")
    theatres = soup.findAll('div', class_ = "col-md-12 cinema_card")
    for theatre in theatres:
        name = tab_delete(theatre.findAll('h3', class_ = "title movie_card_title")[0].text)
        metro = [tab_delete(i) for i in theatre.findAll('span', class_ = "sub_title")[1] if ('\n' in i) and (len(i) > 2)]
        address = tab_delete(theatre.findAll('span', class_ = "sub_title")[0].text)
        phone = 'unknown'
        values = [2, name, address, ', '.join(metro), phone]
        cursor.execute("insert into cinema_halls (brand_id, name, address, metro, phone) values ({},'{}','{}','{}','{}')".format(*values))
    conn.commit()
else:
    print('I think that page is not found')

In [24]:
r2 = requests.get(url_films)
if r2.status_code == 200:
    soup = BeautifulSoup(r2.text, "html.parser")
    films = soup.findAll('div', class_ = "col-md-2 col-sm-6 col-xs-12 movie_card")
    for film in films:
        count = 0
        name = tab_delete_name(film.findAll('span', class_ = "movie_card_header title")[0].text)
        name_2 = cursor.execute('select name from cinemas').fetchall()
        for i in name_2:
            if name in i[0]:
                count += 1
        if count == 0:
            try:
                duration = tab_delete(film.findAll('span', class_ = "hide-md title")[0].text)
            except IndexError:
                duration = '0 мин'
            genres = [tab_delete(i.text) for i in film.findAll('span', class_ = "hidden", itemprop = "genre")]
            values = [name, duration.split()[0], ', '.join(genres)]
            try:
                cursor.execute("insert into cinemas (name, duration, genres) values ('{}', {},'{}')".format(*values))
            except sqlite3.OperationalError:
                continue
    conn.commit()
else:
    print('I think that page is not found')

In [36]:
cinema_halls_cinema_park = ['/belaya-dacha', '/butovo-mall', '/waypark', '/global-city', '/gorizont', '/evropa', '/zelenopark', '/kaluzhskij', '/kutuzovskiy', '/ladoga', '/lefortovo', '/metropolis', '/michurinsky', '/mozhayka', '/oblaka', '/polezhaevskiy', '/5avenu', '/rivera', '/semenovsky', '/city', '/tepliy-stan', '/filion', '/mega-himki', '/cdm', '/chertanovo']

In [37]:
k = 0
hall_id = 17
cinema_id = 0
for theatre in theatres:
    url_table = url_theatres + cinema_halls_cinema_park[k]
    k += 1
    r3 = requests.get(url_table)
    if r3.status_code == 200:
        dates = []
        soup = BeautifulSoup(r3.text, "html.parser")
        date_parser = soup.findAll('input', class_ = "single_card_date_input")
        for date in date_maker(str(date_parser)).split(', '):
            if date not in dates:
                dates.append(date)
        for date in dates:
            url_date = url_table + '/?date=' + date
            r4 = requests.get(url_date)
            if r4.status_code == 200:
                schedule = BeautifulSoup(r4.text, "html.parser")
                films = schedule.findAll('div', class_ = 'shedule_movie')
                for film in films:
                    money = []
                    n = 0
                    title = film.findAll('span', class_ = "movie_card_header title")
                    name = tab_delete_name(title[0].text)
                    cinema_id = try_to_get_cinema_id(name, cursor.execute('select * from cinemas').fetchall())
                    sessions = film.findAll('span', class_ = "shedule_session_time")
                    prices = film.findAll('span', class_ = "shedule_session_price")
                    for price in prices:
                        money.append(number(price.text))
                    for session in sessions:
                        time = tab_delete(session.text)
                        try:
                            values = [cinema_id, hall_id, name, date, time, money[n]]
                        except IndexError:
                            values = [cinema_id, hall_id, name, date, time, 0]
                        n += 1
                        try:
                            cursor.execute("insert into sessions (cinema_id, hall_id, name, date, time, price) values ({},{},'{}','{}', '{}', {})".format(*values))
                        except sqlite3.OperationalError:
                            continue
                        conn.commit()
            else:
                print('I think that page is not found')
    else:
        print('I think that page is not found')
    hall_id += 1 

In [38]:
url_almaz1 = "https://almazcinema.ru/msk/cinema/41/"
url_almaz2 = "https://almazcinema.ru/msk/cinema/40/"

In [39]:
#cursor.execute('drop table sessions')

In [40]:
def metro_address(name):
    lst = name.split('м. ')
    return lst

In [42]:
r5 = requests.get(url_almaz1)
if r5.status_code == 200:
    soup = BeautifulSoup(r5.text, "html.parser")
    name = soup.findAll('h1', class_ = 'page__title')[0].text
    info = tab_delete_name(soup.findAll('div', class_ = 'cinema__address')[0].text)
    phone = soup.findAll('div', class_ = 'cinema__phone')[0].text
    metro = metro_address(info)[-1]
    address = metro_address(info)[0]
    values = [3, name, address, metro, phone]
    cursor.execute("insert into cinema_halls (brand_id, name, address, metro, phone) values ({},'{}','{}','{}','{}')".format(*values))
    conn.commit()
else:
    print('I think that page is not found')

In [43]:
r6 = requests.get(url_almaz2)
if r6.status_code == 200:
    soup = BeautifulSoup(r6.text, "html.parser")
    name = soup.findAll('h1', class_ = 'page__title')[0].text
    info = tab_delete_name(soup.findAll('div', class_ = 'cinema__address')[0].text)
    phone = soup.findAll('div', class_ = 'cinema__phone')[0].text
    metro = metro_address(info)[-1]
    address = metro_address(info)[0]
    values = [3, name, address, metro, phone]
    cursor.execute("insert into cinema_halls (brand_id, name, address, metro, phone) values ({},'{}','{}','{}','{}')".format(*values))
    conn.commit()
else:
    print('I think that page is not found')

In [46]:
url_schedule1 = 'https://almazcinema.ru/msk/cinema/40/schedule/'
url_schedule2 = 'https://almazcinema.ru/msk/cinema/41/schedule/'

In [47]:
def number(price):
    pattern = re.compile(r'[0-9]+[:]?[0-9]+')
    res = ' '.join(pattern.findall(price))
    return res

def tab_delete(name):
    pattern = re.compile(r'[А-Яа-яёЁ0-9.,-:II]+')
    res = ' '.join(pattern.findall(name))
    return res

In [49]:
hall_id = 42
k = 0
r8 = requests.get(url_schedule1)
if r8.status_code == 200:
    soup = BeautifulSoup(r8.text, "html.parser")
    dates = soup.findAll('div', class_ = 'item day')
    for date in dates:
        film_k = 0
        day = (datetime.date.today()+ datetime.timedelta(days=k)).strftime('%Y-%m-%d')
        names = date.findAll('h3')
        for film in names:
            name = tab_delete(str(names[film_k]))[2:-4]
            cinema_id = try_to_get_cinema_id(name, cursor.execute('select * from cinemas').fetchall())
            sessions = date.findAll('a', class_ = 'btn')
            for session in sessions:
                info = session.findAll('span')
                info = number(str(info))
                time = info.split()[0]
                price = info.split()[-1]
                values = [cinema_id, hall_id, name, day, time, price]
                try:
                    cursor.execute("insert into sessions (cinema_id, hall_id, name, date, time, price) values ({},{},'{}','{}', '{}', {})".format(*values))
                except sqlite3.OperationalError:
                    continue
                conn.commit()
            film_k += 1
        k += 1

In [50]:
hall_id = 43
k = 0
r9 = requests.get(url_schedule1)
if r9.status_code == 200:
    soup = BeautifulSoup(r9.text, "html.parser")
    dates = soup.findAll('div', class_ = 'item day')
    for date in dates:
        film_k = 0
        day = (datetime.date.today()+ datetime.timedelta(days=k)).strftime('%Y-%m-%d')
        names = date.findAll('h3')
        for film in names:
            name = tab_delete(str(names[film_k]))[2:-4]
            cinema_id = try_to_get_cinema_id(name, cursor.execute('select * from cinemas').fetchall())
            sessions = date.findAll('a', class_ = 'btn')
            for session in sessions:
                info = session.findAll('span')
                info = number(str(info))
                time = info.split()[0]
                price = info.split()[-1]
                values = [cinema_id, hall_id, name, day, time, price]
                try:
                    cursor.execute("insert into sessions (cinema_id, hall_id, name, date, time, price) values ({},{},'{}','{}', '{}', {})".format(*values))
                except sqlite3.OperationalError:
                    continue
                conn.commit()
            film_k += 1
        k += 1